In [1]:
import sys
import os
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')
os.chdir('c:\\Users\\Danie\\OneDrive\\Documents\\Coding\\pysystemtrade\\pysystemtrade\\examples\\introduction')
# Add the parent directory of pysystemtrade to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [3]:
# Filter the results to a specific date range
start_date = '2002-01-02'
end_date = '2019-12-31'

In [ ]:
import logging

# Set logging level to INFO to suppress DEBUG messages
logging.getLogger().setLevel(logging.INFO)

# Import systemtest
from systems.dt.systemtest import systemtest
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData

# Initialize the data
data = csvFuturesSimData()

# Create and run the system with gov instruments
my_system = systemtest(
    data=data
)

In [ ]:
# View portfolio performance
print(my_system.accounts.portfolio().sharpe())

# Look at positions for a specific instrument
print(my_system.portfolio.get_notional_position("US10").tail())

# Plot cumulative returns
my_system.accounts.portfolio().curve().plot()

In [ ]:
portfolio = my_system.accounts.portfolio()

In [ ]:
portfolio.percent.stats()
portfolio.percent.curve()


In [ ]:
# Get account statistics for the chapter 15 system
portfolio.percent.stats()

In [ ]:
# Plot positions for each instrument

import matplotlib.pyplot as plt
import pandas as pd

# Get all instruments
instruments = my_system.get_instrument_list()

# Create a figure with subplots
fig, axes = plt.subplots(len(instruments), 1, figsize=(10, 3*len(instruments)), sharex=True)

# Plot each instrument's position
for i, instrument in enumerate(instruments):
    position = my_system.portfolio.get_notional_position(instrument)
    position.plot(ax=axes[i])
    axes[i].set_title(f'Positions for {instrument}')
    axes[i].set_ylabel('Position')

# Set common x-label
plt.xlabel('Time')
plt.tight_layout()

In [ ]:
# Test with 100% carry strategy
carry_system = systemtest(forecast_weights={"carry": 1.0, "ewmac8": 0.0, "ewmac32": 0.0})

# Test with 100% trend following
trend_system = systemtest(forecast_weights={"carry": 0.0, "ewmac8": 0.5, "ewmac32": 0.5})

# Test with balanced weights
balanced_system = systemtest(forecast_weights={"carry": 0.33, "ewmac8": 0.33, "ewmac32": 0.34})

In [ ]:
# Print Sharpe ratios for each system
print("Default System Portfolio Sharpe Ratio:", my_system.accounts.portfolio().sharpe())
print("Carry-only System Portfolio Sharpe Ratio:", carry_system.accounts.portfolio().sharpe())
print("Trend-only System Portfolio Sharpe Ratio:", trend_system.accounts.portfolio().sharpe())
print("Balanced System Portfolio Sharpe Ratio:", balanced_system.accounts.portfolio().sharpe())

In [ ]:
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define weight combinations to test
carry_weights = np.arange(0, 1.1, 0.2)
ewmac8_weights = np.arange(0, 1.1, 0.2)

# Store results
results = []

# Temporarily disable logging
original_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.ERROR)

# Iterate through different weight combinations
for carry_weight in carry_weights:
    for ewmac8_weight in ewmac8_weights:
        # Make sure weights sum to 1
        ewmac32_weight = 1.0 - carry_weight - ewmac8_weight
        
        # Skip invalid combinations (where weights would be negative)
        if ewmac32_weight < 0:
            continue
            
        # Round to avoid floating point issues
        ewmac32_weight = round(ewmac32_weight, 2)
        
        # Create system with these weights
        weights = {
            "carry": carry_weight,
            "ewmac8": ewmac8_weight,
            "ewmac32": ewmac32_weight
        }
        
        test_system = systemtest(forecast_weights=weights)
        test_portfolio = test_system.accounts.portfolio()
        sharpe = test_portfolio.sharpe()
        
        # Store results
        results.append({
            "carry": carry_weight,
            "ewmac8": ewmac8_weight,
            "ewmac32": ewmac32_weight,
            "sharpe": sharpe
        })

# Restore original logging level
logging.getLogger().setLevel(original_level)

# Convert results to DataFrame
results_df = pd.DataFrame(results)
print(f"Tested {len(results_df)} weight combinations")

# Find best combination
best_result = results_df.loc[results_df['sharpe'].idxmax()]
print(f"Best weights: carry={best_result['carry']}, ewmac8={best_result['ewmac8']}, ewmac32={best_result['ewmac32']}")
print(f"Best Sharpe: {best_result['sharpe']:.4f}")

# Create a system with the best weights for further analysis
test_system = systemtest(forecast_weights={
    "carry": best_result['carry'],
    "ewmac8": best_result['ewmac8'],
    "ewmac32": best_result['ewmac32']
})
test_portfolio = test_system.accounts.portfolio()

In [ ]:
test_system = systemtest(forecast_weights={"carry": 0.2, "ewmac8": 0.6, "ewmac32": 0.2})


In [18]:
import logging
# Temporarily disable logging
original_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.ERROR)

test_portfolio = test_system.accounts.portfolio()

# Restore original logging level
logging.getLogger().setLevel(original_level)

In [ ]:
test_portfolio.percent.stats()

In [ ]:
# Display portfolio statistics
print("Portfolio Statistics:")
print(f"Sharpe Ratio: {test_portfolio.sharpe():.4f}")
print(f"Annual Return: {test_portfolio.percent.ann_mean():.4f}")
print(f"Annual Std Dev: {test_portfolio.percent.ann_std():.4f}")



# Display PnL
print("\nPortfolio PnL:")
pnl = test_portfolio.curve()
print(pnl.tail())

# Plot cumulative returns
test_portfolio.curve().plot(figsize=(12, 6), title="Test System Cumulative Returns")

In [ ]:
# Display positions for each instrument through time
import matplotlib.pyplot as plt

# Get list of instruments in the system
instruments = test_system.get_instrument_list()

if instruments:
    plt.figure(figsize=(12, 8))
    
    # Plot positions for each instrument
    for instrument in instruments:
        position = test_system.portfolio.get_notional_position(instrument_code=instrument)
        if not position.empty:
            position.plot(label=instrument)
    
    plt.title('Notional Positions Through Time')
    plt.xlabel('Date')
    plt.ylabel('Position Size')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No instruments found in the system")

In [ ]:
# Display positions for each instrument through time, split by decade
import matplotlib.pyplot as plt
import pandas as pd

# Get list of instruments in the system
instruments = test_system.get_instrument_list()

if instruments:
    # Get all positions
    all_positions = {}
    for instrument in instruments:
        position = test_system.portfolio.get_notional_position(instrument_code=instrument)
        if not position.empty:
            all_positions[instrument] = position
    
    if all_positions:
        # Get the full date range
        all_dates = pd.concat([pos for pos in all_positions.values()]).index
        min_date = all_dates.min()
        max_date = all_dates.max()
        
        # Create decade boundaries
        start_decade = (min_date.year // 10) * 10
        end_decade = (max_date.year // 10 + 1) * 10
        decades = [(start_decade + i*10, start_decade + (i+1)*10) for i in range((end_decade - start_decade) // 10)]
        
        # Plot each decade separately
        for decade_start, decade_end in decades:
            decade_start_date = pd.Timestamp(f"{decade_start}-01-01")
            decade_end_date = pd.Timestamp(f"{decade_end}-01-01")
            
            # Filter data for this decade
            decade_data = {}
            for instrument, position in all_positions.items():
                decade_position = position.loc[(position.index >= decade_start_date) & 
                                              (position.index < decade_end_date)]
                if not decade_position.empty:
                    decade_data[instrument] = decade_position
            
            if decade_data:
                plt.figure(figsize=(12, 8))
                for instrument, position in decade_data.items():
                    position.plot(label=instrument)
                
                plt.title(f'Notional Positions {decade_start}-{decade_end}')
                plt.xlabel('Date')
                plt.ylabel('Position Size')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
    else:
        print("No position data available for any instruments")
else:
    print("No instruments found in the system")

In [ ]:
test_portfolio = test_system.accounts.portfolio()
carry_portfolio = carry_system.accounts.portfolio()
trend_portfolio = trend_system.accounts.portfolio()
balanced_portfolio = balanced_system.accounts.portfolio()

In [ ]:
# Print Sharpe ratios for each system
print("Test System Portfolio Sharpe Ratio:", test_portfolio.sharpe())
print("Carry-only System Portfolio Sharpe Ratio:", carry_portfolio.sharpe())
print("Trend-only System Portfolio Sharpe Ratio:", trend_portfolio.sharpe())
print("Balanced System Portfolio Sharpe Ratio:", balanced_portfolio.sharpe())